In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from trsm import TRSM, combos_6j

In [3]:
import os
os.getcwd()

'/uscms_data/d3/srosenzw/workarea/higgs/sixb_analysis/CMSSW_10_2_18/src/sixb/6jet_classifier/analyzers'

In [4]:
import numpy as np
import awkward as ak

In [5]:
%matplotlib widget
import matplotlib.pyplot as plt
from consistent_plots import hist, hist2d

In [23]:
# filename = 'root://cmseos.fnal.gov//eos/uscms/store/user/ekoenig/6BAnalysis/NTuples/2018/SR/QCD/QCD_bEnriched_HT300to500_TuneCP5_13TeV-madgraph-pythia8/ntuple.root'
filename1 = '../../qcd.root'
filename2 = '../../qcd_benriched.root'
trsm_set1 = TRSM(filename=filename1)
trsm_set2 = TRSM(filename=filename2)

In [45]:
import uproot
t1 = uproot.open(filename1 + ":sixBtree")
t2 = uproot.open(filename2 + ":sixBtree")

In [77]:
len(t1['X_m'].array())

4709

In [78]:
len(t2['X_m'].array())

17238

In [17]:
tag = '20210816_5btag_req'

In [43]:
combos7_qcd  = combos_6j(trsm_set1, 7)
combos7_qcdb = combos_6j(trsm_set2, 7)

# scale by total events passing over events in file times cross section, then add histograms together
# can scale by chunks of events

100%|██████████████████████████████████████████████████████████| 1323/1323 [00:02<00:00, 514.81it/s]
ic| combo_p4.pt: <Array [[345, 331, 156, ... 81.8, 50, 28.7]] type='9261 * var * float64'>
ic| type(combo_p4.pt): <class 'awkward.highlevel.Array'>


Total events chosen: 1323


100%|██████████████████████████████████████████████████████████| 5159/5159 [00:10<00:00, 506.85it/s]
ic| combo_p4.pt: <Array [[301, 141, 109, ... 50.1, 40.5, 32.2]] type='36113 * var * float64'>
ic| type(combo_p4.pt): <class 'awkward.highlevel.Array'>


Total events chosen: 5159


In [25]:
combos7_qcd.apply_6j_model(tag)
scores_7_qcd = combos7_qcd.scores_combo

combos7_qcdb.apply_6j_model(tag)
scores_7_qcdb = combos7_qcdb.scores_combo

In [26]:
combos7_qcd.select_highest_scoring_combos()
combos7_qcdb.select_highest_scoring_combos()

100%|█████████████████████████████████████████████████████████| 5159/5159 [00:00<00:00, 6944.43it/s]


In [27]:
evt_score_mask = combos7_qcd.high_score_combo_mask
evt_high_score = combos7_qcd.evt_highest_score

evt_score_mask_b = combos7_qcdb.high_score_combo_mask
evt_high_score_b = combos7_qcdb.evt_highest_score

In [41]:
fig, ax = plt.subplots()
# n, edges, _ = hist(ax, [evt_high_score, evt_high_score_b], histtype='stepfilled', alpha=0.2, density=True)
n, edges, _ = hist(ax, [evt_high_score, evt_high_score_b], histtype='step', color=['C0', 'C1'], label=['QCD', 'QCD, b enriched'], density=True)
ax.set_xlabel('Highest Score of Combinations in Events')
ax.set_ylabel('Number of QCD Combinations Assigned This Score')

ax.legend()
ax.set_title('QCD Background')

fig.savefig('qcd_score.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [55]:
n[0].shape

(100,)

In [57]:
x = (edges[1:] + edges[:-1])/2
sum(n[1][x > 0.8])/sum(n[1])

0.17677846062574146

In [13]:
def norm_hist(arr, bins=100):
    n, b = np.histogram(arr, bins=bins)
    x = (b[:-1] + b[1:]) / 2
    
    return n/n.max(), b, x

In [26]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6,4))

ax.set_title(r"$^7C_6$ Combinations Normalized")

n_7, b_7, x_7 = norm_hist(scores_7)

hist(ax, x_7, weights=n_7, bins=b_7, label='All 6-jet combos')
ax.legend(fontsize='small', loc=9)

ax.set_xlabel('Assigned Score')
ax.set_ylabel('AU')

textstr = f'Entries = {len(scores_7)}'
props = dict(boxstyle='round', facecolor='white', alpha=1)
ax.text(0.8, 1.02, textstr, transform=ax.transAxes, fontsize=9,
        verticalalignment='top', bbox=props)

plt.tight_layout()

fig.savefig('qcd_score.pdf')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [58]:
from matplotlib import ticker

In [65]:
pt_7  = combos7_qcd.combo_features[:,0:6]
eta_7 = combos7_qcd.combo_features[:,6:12]
phi_7 = combos7_qcd.combo_features[:,12:18]
m_7   = np.ones(pt_7.shape)*4

In [72]:
bpt_7  = combos7_qcdb.combo_features[:,0:6]
beta_7 = combos7_qcdb.combo_features[:,6:12]
bphi_7 = combos7_qcdb.combo_features[:,12:18]
bm_7   = np.ones(bpt_7.shape)*4

In [66]:
import vector

In [67]:
p4_7_0 = vector.obj(pt=pt_7[:,0], eta=eta_7[:,0], phi=phi_7[:,0], mass=m_7[:,0])
p4_7_1 = vector.obj(pt=pt_7[:,1], eta=eta_7[:,1], phi=phi_7[:,1], mass=m_7[:,1])
p4_7_2 = vector.obj(pt=pt_7[:,2], eta=eta_7[:,2], phi=phi_7[:,2], mass=m_7[:,2])
p4_7_3 = vector.obj(pt=pt_7[:,3], eta=eta_7[:,3], phi=phi_7[:,3], mass=m_7[:,3])
p4_7_4 = vector.obj(pt=pt_7[:,4], eta=eta_7[:,4], phi=phi_7[:,4], mass=m_7[:,4])
p4_7_5 = vector.obj(pt=pt_7[:,5], eta=eta_7[:,5], phi=phi_7[:,5], mass=m_7[:,5])

In [73]:
bp4_7_0 = vector.obj(pt=bpt_7[:,0], eta=beta_7[:,0], phi=bphi_7[:,0], mass=bm_7[:,0])
bp4_7_1 = vector.obj(pt=bpt_7[:,1], eta=beta_7[:,1], phi=bphi_7[:,1], mass=bm_7[:,1])
bp4_7_2 = vector.obj(pt=bpt_7[:,2], eta=beta_7[:,2], phi=bphi_7[:,2], mass=bm_7[:,2])
bp4_7_3 = vector.obj(pt=bpt_7[:,3], eta=beta_7[:,3], phi=bphi_7[:,3], mass=bm_7[:,3])
bp4_7_4 = vector.obj(pt=bpt_7[:,4], eta=beta_7[:,4], phi=bphi_7[:,4], mass=bm_7[:,4])
bp4_7_5 = vector.obj(pt=bpt_7[:,5], eta=beta_7[:,5], phi=bphi_7[:,5], mass=bm_7[:,5])

In [74]:
p4_7 = p4_7_0 + p4_7_1 + p4_7_2 + p4_7_3 + p4_7_4 + p4_7_5
bp4_7 = bp4_7_0 + bp4_7_1 + bp4_7_2 + bp4_7_3 + bp4_7_4 + bp4_7_5

In [76]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10,4))

ax[0].set_title("Signal 7 Combos")
ax[1].set_title("Background 7 Combos")

n, xedges, yedges, imb = hist2d(ax[1], p4_7.mass[~combos7.sgnl_mask], scores_7[~combos7.sgnl_mask], xbins=np.linspace(0,2000,100))

plt.colorbar(imb, ax=ax[1])

ax[0].set_xlabel('Invariant Mass of 6-jet System [GeV]')
ax[1].set_xlabel('Invariant Mass of 6-jet System [GeV]')
ax[0].set_ylabel('Assigned Score')
ax[1].set_ylabel('Assigned Score')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
score_bins = np.arange(0,1.01,0.01)

In [22]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10,4))

ax[0].set_title("Signal 7 Combos")
ax[1].set_title("Background 7 Combos")

n, b, _ = hist(ax[1], p4_7.mass[~combos7.sgnl_mask], bins=np.linspace(400,2000,100), label='No cut', color='lightcoral')
n, b, _ = hist(ax[1], p4_7.mass[~combos7.sgnl_mask][scores_7[~combos7.sgnl_mask] > 0.8], bins=np.linspace(400,2000,100), label='score > 0.8', color='maroon')


ax[0].set_xlabel('Invariant Mass of 6-jet System [GeV]')
ax[1].set_xlabel('Invariant Mass of 6-jet System [GeV]')
ax[0].set_ylabel('Count')
ax[1].set_ylabel('Count')
ax[0].legend()
ax[1].legend()

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.


In [23]:
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(6,5), sharex=True, gridspec_kw={'height_ratios':[3,1]})

nb, b, _ = hist(ax[0], p4_7.mass[~combos7.sgnl_mask][scores_7[~combos7.sgnl_mask] > 0.8], bins=np.linspace(400,2000,100), label='bkgd score > 0.8', color='maroon')
# n, xedges, yedges, imb = hist2d(ax[1], p4_7.mass[~combos7.sgnl_mask], scores_7[~combos7.sgnl_mask], xbins=np.linspace(0,2000,100))

x = (b[1:] + b[:-1])/2

ax[1].plot(x, ns/nb)
ax[1].plot(x, np.ones_like(x), '--k', alpha=0.2)

ax[1].set_xlabel('Invariant Mass of 6-jet System [GeV]')
ax[0].set_ylabel('Count')
ax[0].legend()

ax[1].set_ylabel(r'Ratio $S/B_C$')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'ns' is not defined

In [ ]:
# plt.close('all')

In [ ]:
# combos7.create_pairs(tag, score_cut = 0.8)
# combos7.create_pairs(tag)

In [ ]:
# combos7.pair_features.shape

In [ ]:
# combos7.apply_2j_model('param_9_selected_events')

In [ ]:
# combos7.scores_pairs[:,0]

In [ ]:
# combos7.pair_target

In [ ]:
# np.savez("../../inputs_2jet_train.npz", inputs=combos7.pair_features, targets=combos7.pair_target)

In [ ]:
# pairs = pairs_2j(combos7)

In [ ]:
# combos7.pair_target[combos7.pair_target > -1]

In [ ]:
# fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10,3))

# ax = axs[0]
# ax.set_title(r"$^7C_6$ Combinations with Pairing, Normalized", pad=10)

# bins = np.linspace(0,1.01,100)

# scores_2_noscorecut = combos7.scores_pairs[:,0]


# n_2_all, b_2_all, x_2_all = norm_hist(scores_2_noscorecut)
# c_2_all, b_2_all, x_2_all = norm_hist(scores_2_noscorecut[combos7.pair_target > -1])
# w_2_all, b_2_all, x_2_all = norm_hist(scores_2_noscorecut[combos7.pair_target == -1])

# hist(ax, x_2_all, weights=n_2_all, bins=b_2_all, label='All pairs')
# hist(ax, x_2_all, weights=c_2_all, bins=b_2_all, label='Correct pairs')
# hist(ax, x_2_all, weights=w_2_all, bins=b_2_all, label='Incorrect pairs')
# ax.legend(fontsize='small', loc=9)

# ax.set_xlabel('Assigned Score')
# ax.set_ylabel('AU')

# textstr = f'Entries = {len(scores_2_noscorecut)}'
# props = dict(boxstyle='round', facecolor='white', alpha=1)
# ax.text(0.8, 1.02, textstr, transform=ax.transAxes, fontsize=9,
#         verticalalignment='top', bbox=props)

# ax = axs[1]
# ax.set_title(r"$^7C_6$ Combinations with Pairing", pad=10)

# formatter = ticker.ScalarFormatter(useMathText=True)
# formatter.set_scientific(True) 
# formatter.set_powerlimits((-1,1)) 
# ax.yaxis.set_major_formatter(formatter) 

# n_2_all, b_2_all = np.histogram(scores_2_noscorecut, bins=100)
# c_2_all, b_2_all = np.histogram(scores_2_noscorecut[combos7.pair_target > -1], bins=100)
# w_2_all, b_2_all = np.histogram(scores_2_noscorecut[combos7.pair_target == -1], bins=100)

# x_2_all = (b_2_all[1:] + b_2_all[:-1]) / 2

# # hist(ax, x_2_all, weights=n_2_all, bins=b_2_all, label='All 6-jet combos')
# hist(ax, x_2_all, weights=c_2_all, bins=b_2_all, label='Correct pairs')
# hist(ax, x_2_all, weights=w_2_all, bins=b_2_all, label='Incorrect pairs')
# ax.legend(fontsize='small', loc=9)

# ax.set_xlabel('Assigned Score')
# ax.set_ylabel('Number of Combinations Per Bin')

# textstr = f'Entries = {len(scores_2_noscorecut)}'
# props = dict(boxstyle='round', facecolor='white', alpha=1)
# ax.text(0.8, 1.02, textstr, transform=ax.transAxes, fontsize=9,
#         verticalalignment='top', bbox=props)

# ax.text(0.3, 0.25, f'Ratio correct/total above 0.8 = {100*sum(c_2_all[x_2_all > 0.8])/sum(n_2_all[x_2_all > 0.8]):.0f}%', transform=ax.transAxes)
# # ax.ticklabel_format(useMathText=True, useOffset=True)

# plt.tight_layout()
# plt.show()

In [ ]:
# fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10,3))

# ax = axs[0]
# ax.set_title(r"$^7C_6$ Combinations with Pairing, Normalized")

# bins = np.linspace(0,1.01,100)

# scores_2 = combos7.scores_pairs[:,0]


# n_7, b_7, x_7 = norm_hist(scores_2)
# c_n_7, b_7, x_7 = norm_hist(scores_2[combos7.pair_target > -1])
# w_n_7, b_7, x_7 = norm_hist(scores_2[combos7.pair_target == -1])

# hist(ax, x_7, weights=n_7, bins=b_7, label='All pairs')
# hist(ax, x_7, weights=c_n_7, bins=b_7, label='Correct pairs')
# hist(ax, x_7, weights=w_n_7, bins=b_7, label='Incorrect pairs')
# ax.legend(fontsize='small', loc=9)

# ax.set_xlabel('Assigned Score')
# ax.set_ylabel('AU')

# textstr = f'Entries = {len(scores_2)}'
# props = dict(boxstyle='round', facecolor='white', alpha=1)
# ax.text(0.8, 1.02, textstr, transform=ax.transAxes, fontsize=9,
#         verticalalignment='top', bbox=props)

# ax = axs[1]
# ax.set_title(r"$^7C_6$ Combinations with Pairing")

# n_7, b_7 = np.histogram(scores_2, bins=100)
# c_n_7, b_7 = np.histogram(scores_2[combos7.pair_target > -1], bins=100)
# w_n_7, b_7 = np.histogram(scores_2[combos7.pair_target == -1], bins=100)

# x_7 = (b_7[1:] + b_7[:-1]) / 2

# # hist(ax, x_7, weights=n_7, bins=b_7, label='All 6-jet combos')
# hist(ax, x_7, weights=c_n_7, bins=b_7, label='Correct pairs')
# hist(ax, x_7, weights=w_n_7, bins=b_7, label='Incorrect pairs')
# ax.legend(fontsize='small', loc=9)

# ax.set_xlabel('Assigned Score')
# ax.set_ylabel('Number of Combinations Per Bin')

# textstr = f'Entries = {len(scores_2)}'
# props = dict(boxstyle='round', facecolor='white', alpha=1)
# ax.text(0.8, 1.02, textstr, transform=ax.transAxes, fontsize=9,
#         verticalalignment='top', bbox=props)

# ax.text(0.3, 50000, f'Ratio correct/total above 0.8 = {100*sum(c_n_7[x_7 > 0.8])/sum(n_7[x_7 > 0.8]):.0f}%')

# plt.tight_layout()
# plt.show()